In [23]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import col, count, when, isnull, create_map, lit
from itertools import chain

spark = SparkSession.builder.getOrCreate()
file_path = "Files/online_shoppers_intention.csv"

# Lecture du fichier
df = spark.read.csv(
    file_path,
    header=True,
    inferSchema=True
)

# Sauvegarde en couche Bronze
df.write.mode("overwrite").format("delta").saveAsTable("bronze_raw_shopping")

print(f"Table Bronze creee: {df.count()} lignes chargees.")

StatementMeta(, 122dcaa7-74a8-4a88-abdf-45139ecae883, 25, Finished, Available, Finished)

Table Bronze creee: 12330 lignes chargees.


In [24]:
# Analyser les colonnes
any_missing = False
for column in df.columns:
    # On utilise isnull() qui est plus robuste pour tous les types
    missing_count = df.filter(col(column).isNull()).count()
    
    if missing_count > 0:
        any_missing = True
        pct_missing = (missing_count / total_count) * 100
        print(f"   {column}: {missing_count} manquants ({pct_missing:.2f}%)")

if not any_missing:
    print("  Aucune valeur manquante détectée dans le dataset.")

StatementMeta(, 122dcaa7-74a8-4a88-abdf-45139ecae883, 26, Finished, Available, Finished)

  Aucune valeur manquante détectée dans le dataset.


In [25]:
# Conversion des colonnes Booleennes et String en format exploitable
df_silver = df.withColumn("Revenue_label", when(col("Revenue") == True, 1).otherwise(0)) \
              .withColumn("Weekend_label", when(col("Weekend") == True, 1).otherwise(0))

StatementMeta(, 122dcaa7-74a8-4a88-abdf-45139ecae883, 27, Finished, Available, Finished)

In [26]:
# Encodage du mois en valeur numerique pour le ML
month_map = {
    'Feb': 2, 'Mar': 3, 'May': 5, 'June': 6, 'Jul': 7,
    'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12
}

# Transformation du dictionnaire en colonnes Spark pour create_map
mapping_expr = create_map([lit(x) for x in chain(*month_map.items())])

df_silver = df_silver.withColumn("Month_num", mapping_expr[col("Month")])

# Sauvegarde ciblée dans le schéma 'silver'
df_silver.write.mode("overwrite").format("delta").saveAsTable("silver.cleaned_shopping_data")

print("Table nettoyée et sauvegardée avec succès dans le schéma 'silver'.")

StatementMeta(, 122dcaa7-74a8-4a88-abdf-45139ecae883, 28, Finished, Available, Finished)

Table nettoyée et sauvegardée avec succès dans le schéma 'silver'.
